In [1]:
import math
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
import random

In [2]:
xor_dict={(0,0):0,
          (0,1):1,
          (1,0):1,
          (1,1):0}
df=pd.DataFrame({0:[(0,0),(0,1),(1,0),(1,1)],
   1:[0,1,1,0]})
df_2=df.sample(n=100000,replace=True,random_state=2022).reset_index(drop=True)

In [238]:
df_2[0][1]

(1, 1)

In [3]:
# sigmoid & sigmoid prime
def sig(x):
    return (1/(1+np.exp(-x)))

def sig_prime(x):
    return (x * (1 - x))

In [4]:
# # error function
# def mse(weights,df=xor_dict):
#   # output will be a (3,1) np.array
#   # last one is mse of pred_y, others are mse of hidden layer
#   output=0
#   w=weights.reshape(3,3)
#   for k,v in df.items():
#     x=np.array(list((1,k[0],k[1])))
#     hl=np.array((1,sig(np.inner(w[0],x)),sig(np.inner(w[1],x)))) # hidden layer
#     nodes=np.vstack((x,hl))
#     # y_hat=sig(np.inner(w[2],hl))
#     if sig(np.inner(w[2],hl))>=0.5:
#       output += (v-1)**2
#     else:
#       output += (v-0)**2
#   # print(output)
#   return(output/4)

In [112]:
def grdmse(w, nodes, y_hat, diff, lr): 
  dlt=np.zeros(shape=(3,3)) 
  sp_y=sig_prime(y_hat)
  sp_s1,sp_s2=sig_prime(nodes[1,1:3]) 
  dlt[2]=diff*sp_y*nodes[1]
  dlt[1]=diff*sp_y*w[2,2]*sp_s2*nodes[0]
  dlt[0]=diff*sp_y*w[2,1]*sp_s1*nodes[0]
  w += -lr*dlt
  return(w)

In [118]:
def forward(nodes, weights):
    nodes[1][1] = sig(nodes[0].dot(weights[0].T))
    nodes[1][2] = sig(nodes[0].dot(weights[1].T))
    output = sig(nodes[1].dot(weights[2].T))
    return nodes, output

In [133]:
def train(nodes, target, weights):
    nodes, output = forward(nodes, weights)
    error = output - target
    return grdmse(weights, nodes, output, error, 0.3)

In [139]:
def mse_1(weights):
    re = 0
    nodes = np.ones(6).reshape(2,3)
    for k in xor_dict.keys():
        nodes[0] = np.insert(k, 0 , 1, axis = 0)
        n, output = forward(nodes, weights)
        re += abs(xor_dict[k] - output)
    return re/4

In [28]:
INPUTS = [
        [0,0],
        [0,1],
        [1,0],
        [1,1]
    ]

OUTPUTS = [
        [0],
        [1],
        [1],
        [0]
    ]

In [160]:
nodes = np.ones(6).reshape(2,3)
weights = np.random.uniform(-0.5,0.5,size=(3,3))
for epoch in range(1,10001):
    indexes = [0,1,2,3]
    random.shuffle(indexes)

    for j in indexes:
        nodes[0] = [1, INPUTS[j][0], INPUTS[j][1]]
        weights = train(nodes, OUTPUTS[j][0], weights)
    
    if epoch%1000 == 0:
        cost = 0
        for j in range(4):
            nodes[0] = [1, INPUTS[j][0], INPUTS[j][1]]
            n, o = forward(nodes, weights)
            cost += (OUTPUTS[j][0] - o) ** 2
        cost /= 4
        print("epoch", epoch, "mean squared error:", cost)       
        # print('\n',weights)

for i in range(4):
    nodes[0] = [1, INPUTS[i][0],INPUTS[i][1]]
    n, result = forward(nodes, weights)
    print("for input", INPUTS[i], "expected", OUTPUTS[i][0], "predicted", f"{result:4.4}", "which is", "correct" if round(result)==OUTPUTS[i][0] else "incorrect")

epoch 1000 mean squared error: 0.24951779053021955
epoch 2000 mean squared error: 0.17644670679225186
epoch 3000 mean squared error: 0.01238693108342294
epoch 4000 mean squared error: 0.0038560282339353685
epoch 5000 mean squared error: 0.0021779523165206434
epoch 6000 mean squared error: 0.0014960308458517135
epoch 7000 mean squared error: 0.001131904242654962
epoch 8000 mean squared error: 0.000907047917773129
epoch 9000 mean squared error: 0.0007550215751310009
epoch 10000 mean squared error: 0.0006456631489909071
for input [0, 0] expected 0 predicted 0.02258 which is correct
for input [0, 1] expected 1 predicted 0.9758 which is correct
for input [1, 0] expected 1 predicted 0.9758 which is correct
for input [1, 1] expected 0 predicted 0.03008 which is correct


In [111]:
mse(weights)

0.02813689927374971

In [148]:
or_weights = np.random.uniform(-1,1,size=(3,3))
# or_weights = np.array([-0.70790987, 1.31670139, 1.35362421, -0.70790987, 1.21, 2.35362421, 0.58713112, 1.23913159, -0.83913159]).reshape(3,3)
# e_list = []
print(or_weights)
weights = or_weights
nodes = np.ones(6).reshape(2,3)
for i in range(len(df_2)):
    nodes[0] = np.insert(df_2[0][i], 0 , 1, axis = 0)
    weights = train(nodes, df_2[1][i], weights)

    if mse_1(weights) <= 0.1 :
        print('ITER: ', i)
        break
    # e_list.append(output)
    # e_list.append(df_2[1][i])


[[ 0.30200605 -0.16110675 -0.08595288]
 [ 0.2884714   0.8204488   0.45662969]
 [ 0.30074322 -0.19190217  0.94155538]]
ITER:  7617


In [149]:
or_weights

array([[-4.79028678,  3.11592442,  3.18339207],
       [-2.11206196,  5.46367092,  5.47906304],
       [-2.89733718, -6.91164357,  6.45475589]])

In [150]:
weights

array([[-4.79028678,  3.11592442,  3.18339207],
       [-2.11206196,  5.46367092,  5.47906304],
       [-2.89733718, -6.91164357,  6.45475589]])

In [151]:
mse_1(weights)

0.09995296119405496

In [14]:
df_2.iloc[-20:,:]

,0,1
99980,"(0, 1)",1
99981,"(0, 0)",0
99982,"(0, 0)",0
99983,"(0, 0)",0
99984,"(1, 0)",1
99985,"(1, 1)",0
99986,"(0, 1)",1
99987,"(1, 1)",0
99988,"(1, 0)",1
99989,"(0, 1)",1


In [23]:
e_list[-40:]

NameError: name 'e_list' is not defined

In [107]:
w = np.array([1,1,1,-1,-1,-1,1,1,1]).reshape(3,3)
print(w)

[[ 1  1  1]
 [-1 -1 -1]
 [ 1  1  1]]


In [77]:
INPUTS = [
        [0,0],
        [0,1],
        [1,0],
        [1,1]
    ]

OUTPUTS = [
        [0],
        [1],
        [1],
        [0]
    ]

In [92]:
nodes[0][1], nodes[0][2] = INPUTS[2]
print(nodes)

[[1.         1.         0.        ]
 [1.         0.88079708 0.11920292]]


In [93]:
nodes[1][1] = sig(nodes[0].dot(w[0]))
nodes[1][2] = sig(nodes[0].dot(w[1]))
print(nodes)

[[1.         1.         0.        ]
 [1.         0.88079708 0.11920292]]


In [94]:
output = sig(nodes[1].dot(w[2]))
print(output)

0.8807970779778823


In [99]:
error = output - OUTPUTS[2]
print(error)

[-0.11920292]


In [96]:
w

array([[ 1,  1,  1],
       [-1, -1, -1],
       [ 1,  1,  1]])

In [103]:
nodes

array([[1.        , 1.        , 0.        ],
       [1.        , 0.88079708, 0.11920292]])

In [104]:
w

array([[ 1,  1,  1],
       [-1, -1, -1],
       [ 1,  1,  1]])

In [105]:
print(forward(nodes, w))

0.8807970779778823


In [88]:
def tgrdmse(w, nodes, y_hat, diff): 
  dlt=np.zeros(shape=(3,3)) 
  sp_y=sig_prime(y_hat)
  sp_s1,sp_s2=sig_prime(nodes[1,1:3]) 
  dlt[2]=diff*sp_y*nodes[1]
  dlt[1]=diff*sp_y*w[2,2]*sp_s2*nodes[0]
  dlt[0]=diff*sp_y*w[2,1]*sp_s1*nodes[0]
  return(dlt)

In [101]:
print(tgrdmse(w, nodes, output, error))

[[-0.00131405 -0.00131405 -0.        ]
 [-0.00131405 -0.00131405 -0.        ]
 [-0.01251554 -0.01102365 -0.00149189]]


In [86]:
w

array([[ 1,  1,  1],
       [-1, -1, -1],
       [ 1,  0,  0]])